In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.datasets import make_regression
from sklearn.metrics import *
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost import XGBClassifier, XGBRegressor
from imblearn.datasets import make_imbalance
from category_encoders.target_encoder import TargetEncoder
import statsmodels.api as sm

In [2]:
df = pd.read_csv('df_pipe')

In [3]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,volume,price_bin,price
0,0,0.23,Ideal,E,SI2,61.5,55.0,38.202030,0.0,326.0
1,1,0.21,Premium,E,SI1,59.8,61.0,34.505856,0.0,326.0
2,2,0.23,Good,E,VS1,56.9,65.0,38.076885,0.0,327.0
3,3,0.29,Premium,I,VS2,62.4,58.0,46.724580,0.0,334.0
4,4,0.31,Good,J,SI2,63.3,58.0,51.917250,0.0,335.0


In [4]:
X=df.drop(['price','price_bin'],axis=1)
y1=df.price_bin
y2=df.price

In [5]:
logreg=LogisticRegression(random_state=14)
cross_val_score(logreg, X.select_dtypes(exclude='O'), y1, cv=5, scoring='accuracy').mean()

0.7439624413145539

In [6]:
lab1 = LabelEncoder()
lab2 = LabelEncoder()
lab3 = LabelEncoder()

In [7]:
tab1 = TargetEncoder()
tab2 = TargetEncoder()
tab2 = TargetEncoder()

In [8]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['cut', 'color','clarity']))

In [9]:
pipe = make_pipeline(column_trans, logreg)

In [10]:
cross_val_score(pipe, X, y1, cv=5, scoring='accuracy').mean()

0.7250892018779342

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y1)

In [12]:
def upsample_classes(data, target):
    
    labels = list(data[target].unique())
    i = 0
    lst = []
    while len(lst) < 3:
        lst.append(labels[i])
        i+=1
    
    classes = []
    for c in lst:
        classes.append(data[data[target]==c])
    
    length = 0
    class_lab = None
    for c in classes:
        if len(c)>length:
            length=len(c)
            class_lab = c
    class_lab = class_lab[target].unique()[0]
    
    a = pd.concat(classes)
    maj_class = a[a[target]==class_lab]

    lst.remove(class_lab)
    
    new_classes=[]
    for i in lst:
        new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

    b = pd.concat(new_classes)
    upsample = pd.concat([a[a[target]==class_lab],b])

    return upsample

In [13]:
upsample_train = upsample_classes(pd.concat([X_train,y_train],axis=1),'price_bin')

In [14]:
X_train=upsample_train.drop('price_bin',axis=1)
y_train=upsample_train.price_bin

In [15]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['cut', 'color',
                                                   'clarity'])])),
                ('logisticregression', LogisticRegression(random_state=14))])

In [16]:
p=pipe.predict(X_test)

In [17]:
print(classification_report(p,y_test))

              precision    recall  f1-score   support

         0.0       0.49      0.81      0.61      5757
         1.0       0.48      0.24      0.32      4686
         2.0       0.30      0.09      0.14      2870
         3.0       0.00      0.00      0.00         0

    accuracy                           0.45     13313
   macro avg       0.31      0.29      0.27     13313
weighted avg       0.44      0.45      0.41     13313

